# Load

This notebook is used to demonstrate ingestion of data, refer to cell #2 for configuration values

In [1]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,SOLNDEMOUSR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [2]:
# Parameter initialization

import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import time
import re

pd.set_option('display.max_colwidth', None)

def get_file_from_download_url(p_fl_url):
    splits = p_fl_url.split('/')
    return splits[len(splits) - 1]

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name


DATA_FILE_URL = 'https://priorityhealthtransparencymrfs.s3.amazonaws.com/2023_03_01_priority_health_HMO_in-network-rates.zip'
DATA_FILE = get_file_from_download_url(DATA_FILE_URL)
DATA_FILE = '2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz'

DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
# INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

# This will need to be updated based on provider
# Priority Health ~ 500
# CIGNA ~ 15000
SEGMENTS_PER_TASK = 500

DAG_ROWS = 5
DAG_COLS = 5

warehouses = config['SNOW_CONN']['warehouse']
create_warehouses = True
warehouses_count = 10
#warehouses = 'INDSOL_PRICE_TRANS_TASK_0_WH,INDSOL_PRICE_TRANS_TASK_1_WH,INDSOL_PRICE_TRANS_TASK_2_WH,INDSOL_PRICE_TRANS_TASK_3_WH,INDSOL_PRICE_TRANS_TASK_4_WH,INDSOL_PRICE_TRANS_TASK_5_WH,INDSOL_PRICE_TRANS_TASK_6_WH,INDSOL_PRICE_TRANS_TASK_7_WH,INDSOL_PRICE_TRANS_TASK_8_WH,INDSOL_PRICE_TRANS_TASK_9_WH,INDSOL_PRICE_TRANS_TASK_10_WH'
#warehouses = 'INDSOL_PRICE_TRANS_TASK_0_WH,INDSOL_PRICE_TRANS_TASK_1_WH,INDSOL_PRICE_TRANS_TASK_2_WH,INDSOL_PRICE_TRANS_TASK_3_WH,INDSOL_PRICE_TRANS_TASK_4_WH,INDSOL_PRICE_TRANS_TASK_5_WH'
warehouses_lst = [ f'INDSOL_PRICE_TRANS_TASK_{i}_WH' for i in range(warehouses_count) ]
warehouses = ','.join(warehouses_lst)

# XSMALL | SMALL | MEDIUM | LARGE | XLARGE | XXLARGE | XXXLARGE | X4LARGE | X5LARGE | X6LARGE
warehouse_size = 'MEDIUM'


In [3]:
# Download file and upload to stage

from urllib import request

# Create local download folder
DOWNLOAD_FOLDER=f'{PROJECT_HOME_DIR}/temp'
os.makedirs(DOWNLOAD_FOLDER ,exist_ok=True)
download_file_path = os.path.join(DOWNLOAD_FOLDER, DATA_FILE)

# if not os.path.exists(os.path.dirname(download_file_path)):
#     print(f'Downloading file to local: {download_file_path}')
#     request.urlretrieve(DATA_FILE_URL, download_file_path)

In [4]:
# Upload data file to stage, if not present

sql_stmt = f'''select relative_path
from directory(@{INPUT_DATA_STAGE})
where relative_path like '%{DATA_FILE}%'
;'''
# print(sql_stmt)

sp_session.sql(f'alter stage {INPUT_DATA_STAGE} refresh;').collect()
df = sp_session.sql(sql_stmt).to_pandas()
if (len(df) < 1):
    print(f'Uploading to stage {INPUT_DATA_STAGE} ...')
    sp_session.file.put(
        local_file_name = download_file_path
        ,stage_location = f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}'
        ,auto_compress=False ,overwrite=True)
    
    sp_session.sql('alter stage {INPUT_DATA_STAGE} refresh;').collect()
    df = sp_session.sql(sql_stmt).to_pandas()

display(df)

,RELATIVE_PATH
0,data/2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz


In [9]:
# Create warehouses for parallelism

if create_warehouses == True:
    whs = warehouses.split(',')
    print(f'Creating {len(whs)} warehouses ..')

    sp_session.sql('use role sysadmin;').collect()
    for wh_name in whs:
        print(f'    - {wh_name}')
        sql_stmt = f'''
            create warehouse if not exists  {wh_name} with
                warehouse_size = 'xsmall'
                auto_resume = true
                auto_suspend = 300
                comment = 'warehouse created as part of pricing transperancy industry solution usecase.'
            ;
        '''
        sp_session.sql(sql_stmt).collect()
        rl = config['APP_DB']['role']
        sp_session.sql(f'grant ALL PRIVILEGES on warehouse {wh_name} to role {rl};').collect()
       
    sp_session.use_role(f'''{config['APP_DB']['role']}''')

Creating 10 warehouses ..
    - INDSOL_PRICE_TRANS_TASK_0_WH
    - INDSOL_PRICE_TRANS_TASK_1_WH
    - INDSOL_PRICE_TRANS_TASK_2_WH
    - INDSOL_PRICE_TRANS_TASK_3_WH
    - INDSOL_PRICE_TRANS_TASK_4_WH
    - INDSOL_PRICE_TRANS_TASK_5_WH
    - INDSOL_PRICE_TRANS_TASK_6_WH
    - INDSOL_PRICE_TRANS_TASK_7_WH
    - INDSOL_PRICE_TRANS_TASK_8_WH
    - INDSOL_PRICE_TRANS_TASK_9_WH


In [10]:
# Cleanup block

# We will cleanup specific resources and artifacts from possible previous runs.
stmts = [
    f''' delete from segment_task_execution_status where data_file = '{DATA_FILE}'; '''
    ,f''' delete from task_to_segmentids where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_file_header where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_segment_header where data_file = '{DATA_FILE}'; '''
    ,f''' alter stage {INPUT_DATA_STAGE} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {TARGET_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{TARGET_DATA_STAGE}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

 truncating tables ...
 cleaning up files in external stage under path raw_parsed/2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json/ ...


In [11]:
# Cautious enablement, used during development for testing

print(f'Cleaning dags for datafile: {DATA_FILE_BASENAME_CLEANSED}')
sp_session.call('delete_dag_for_datafile',DATA_FILE_BASENAME_CLEANSED ,False);

Cleaning dags for datafile: 2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json


In [12]:
# reset the warehouse size to desired

print(f'''No of warehouses: {len(warehouses.split(','))}''')
for wh in warehouses.split(','):
    sp_session.sql(f''' alter warehouse {wh} set max_concurrency_level = 8; ''').collect()
    sp_session.sql(f''' alter warehouse {wh} set warehouse_size = {warehouse_size}; ''').collect()


No of warehouses: 10


---
## Data loading
We will be loading the segments and file header using DAG. 

In [13]:
# we build out the DAG
df = sp_session.call('in_network_rates_dagbuilder_matrix' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}' ,DATA_FILE 
    ,f"@{TARGET_DATA_STAGE}/{TARGET_FOLDER}" ,SEGMENTS_PER_TASK ,warehouses ,DAG_ROWS ,DAG_COLS)

sp_session.sql(f''' alter stage {TARGET_DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "data_file": "2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz",
  "root_task": "DAG_ROOT_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json",
  "status": true,
  "task_matrix_shape": [
    5,
    5
  ],
  "term_task": "TERM_tsk_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json"
}


The above operation results in defining the DAG in Snowflake like here. The task names are specific to the data file being parsed.
![](../../doc/soln_images/task_dags.png)

In [14]:
# Next we invoke the DAG

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' execute task DAG_ROOT_{DATA_FILE_BASENAME_CLEANSED}; '''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')

Started at: 04:57:56
 execute task DAG_ROOT_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json; 
Ended at: 04:57:57
Elapsed: 0:00:00.696222


---
## Inspection

In [15]:

print('Tasks to segments')
file_ingestion_df = sp_session.table('TASK_TO_SEGMENTIDS').filter(F.col('DATA_FILE') == F.lit(DATA_FILE)).to_pandas()
display(file_ingestion_df)

Tasks to segments


,BUCKET,DATA_FILE,ASSIGNED_TASK_NAME,FROM_IDX,TO_IDX,SEGMENTS_RECORD_COUNT,INSERTED_AT
0,20,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_10001_10500,10001,10500,499,2023-03-21 01:57:35.127
1,32,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_16001_16500,16001,16500,499,2023-03-21 01:57:35.127
2,4,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_2001_2500,2001,2500,499,2023-03-21 01:57:35.127
3,18,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_9001_9500,9001,9500,499,2023-03-21 01:57:35.127
4,27,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_13501_14000,13501,14000,499,2023-03-21 01:57:35.127
...,...,...,...,...,...,...,...
70,28,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_14001_14500,14001,14500,499,2023-03-21 01:57:35.127
71,12,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_6001_6500,6001,6500,499,2023-03-21 01:57:35.127
72,2,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_1001_1500,1001,1500,499,2023-03-21 01:57:35.127
73,71,2023-01-01_UMR--Inc-_Third-Party-Administrator_ARKANSAS-HEART-HOSPITAL_HAHH_in-network-rates.json.gz,T_2023_01_01_UMR_Inc_Third_Party_Administrator_ARKANSAS_HEART_HOSPITAL_HAHH_in_network_rates_json_35501_36000,35501,36000,499,2023-03-21 01:57:35.127


In [16]:

print('Tasks ,warehouses and state')
sp_session.sql(f''' SHOW TASKS IN  DATABASE {config['APP_DB']['database']}; ''').collect()
stmt = f'''
    select "name" as task_name
        ,"warehouse" as warehouse
        ,"state" as state
    from table(result_scan(last_query_id()))
    where "name" like '%{DATA_FILE_BASENAME_CLEANSED.upper()}%'
       -- and state != 'suspended'
    order by state;
'''
df = sp_session.sql(stmt).to_pandas()
display(df)

Tasks ,warehouses and state


,TASK_NAME,WAREHOUSE,STATE
0,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_37001_999999999,INDSOL_PRICE_TRANS_TASK_5_WH,started
1,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_2501_3000,INDSOL_PRICE_TRANS_TASK_2_WH,started
2,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_26001_26500,INDSOL_PRICE_TRANS_TASK_1_WH,started
3,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_28501_29000,INDSOL_PRICE_TRANS_TASK_2_WH,started
4,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_3001_3500,INDSOL_PRICE_TRANS_TASK_7_WH,started
...,...,...,...
75,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_25001_25500,INDSOL_PRICE_TRANS_TASK_1_WH,started
76,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_25501_26000,INDSOL_PRICE_TRANS_TASK_6_WH,started
77,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_26501_27000,INDSOL_PRICE_TRANS_TASK_6_WH,started
78,T_2023_01_01_UMR_INC_THIRD_PARTY_ADMINISTRATOR_ARKANSAS_HEART_HOSPITAL_HAHH_IN_NETWORK_RATES_JSON_27001_27500,INDSOL_PRICE_TRANS_TASK_1_WH,started


--- 
### Closeout

    With that we are finished this section of the demo setup

In [17]:
sp_session.close()
print('Finished!!!')

Finished!!!
